# AUB Compass — AUB Tutoring & Course Advisor
**Submitted by:** Zainab Saad, 202472448


---

## Create Service Identity Files
### AUB Compass Tutoring Service

In [1]:
# -*- coding: utf-8 -*-
import os, datetime, textwrap
from pathlib import Path
from PIL import Image, ImageDraw, ImageFont

root = Path("/content")
(me := root / "me").mkdir(parents=True, exist_ok=True)
(logs := root / "logs").mkdir(exist_ok=True)

academic_service = {
    "name": "AUB Compass Tutoring Service",
    "mission": "Empower AUB students with personalized course recommendations and tutoring support to make confident academic decisions and excel in their studies.",
    "services": [
        "AI Course Advisor - answers questions about AUB courses, majors, electives, and prerequisites.",
        "Study Planner - helps organize semester schedules and balance workloads.",
        "Skill Path Finder - recommends electives and minors for goals like data science, writing, or communication.",
        "Tutoring Resource Hub - provides study tips and subject guidance.",
        "Feedback Tracker - records common questions to improve service quality."
    ],
    "team": [
        {"name": "Academic Director - Dr. Layla Haddad", "bio": "Oversees tutoring quality and curriculum alignment."},
        {"name": "Technical Lead - Rami Choueiri", "bio": "Responsible for the AUB Compass AI platform and integrations."},
        {"name": "Student Success Lead - Nour Mansour", "bio": "Coordinates tutoring sessions and student onboarding."}
    ],
    "value_prop": [
        "Combines academic expertise with intelligent guidance for fast, accurate advice.",
        "Encourages students to verify choices with official AUB advisors before registration.",
        "Provides consistent, supportive, and ethical tutoring help."
    ],
    "contact": "support@aubcompass.example (fictional)",
    "copyright": "(c) {} AUB Compass Tutoring Service - Fictional service for coursework.".format(datetime.datetime.utcnow().year)
}

def ascii_safe(s):
    for k,v in [("—","-"),("–","-")]:
        s = s.replace(k,v)
    return s

# Build PDF lines
lines = [
    academic_service["name"] + " - Service Profile",
    "",
    "Mission: " + academic_service["mission"],
    "",
    "Services:",
] + ["* " + s for s in academic_service["services"]] + [
    "",
    "Team:",
] + ["* " + m["name"] + ": " + m["bio"] for m in academic_service["team"]] + [
    "",
    "Value Proposition:",
] + ["* " + v for v in academic_service["value_prop"]] + [
    "",
    "Contact: " + academic_service["contact"],
    academic_service["copyright"]
]
lines = [ascii_safe(x) for x in lines]

def save_text_as_pdf(lines, path):
    width, height, margin, line_h = 1240, 1754, 28, 28
    img = Image.new("RGB", (width, height), "white")
    draw = ImageDraw.Draw(img)
    font = ImageFont.load_default()
    y = margin
    for raw in lines:
        for w in textwrap.wrap(raw, width=95) or [""]:
            if y + line_h > height - margin:
                break
            draw.text((margin, y), w, font=font, fill=(0,0,0))
            y += line_h
    img.save(path, "PDF", resolution=150.0)

pdf_path = me / "about_service.pdf"
save_text_as_pdf(lines, str(pdf_path))

# Summary file
summary_lines = [
    academic_service["name"] + " - Summary",
    "",
    "Mission",
    "-------",
    academic_service["mission"],
    "",
    "What We Offer",
    "-------------",
] + ["- " + s for s in academic_service["services"]] + [
    "",
    "Team",
    "----",
] + ["- " + m["name"] + ": " + m["bio"] for m in academic_service["team"]] + [
    "",
    "Value Proposition",
    "-----------------",
] + ["- " + v for v in academic_service["value_prop"]] + [
    "",
    "Contact",
    "-------",
    academic_service["contact"],
    "",
    "(For academic use - fictional tutoring service identity.)",
]
(summary_path := me / "service_summary.txt").write_text("\n".join(summary_lines), encoding="utf-8")

print("Wrote:", pdf_path)
print("Wrote:", summary_path)


/tmp/ipython-input-3347262059.py:31: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  "copyright": "(c) {} AUB Compass Tutoring Service - Fictional service for coursework.".format(datetime.datetime.utcnow().year)


Wrote: /content/me/about_service.pdf
Wrote: /content/me/service_summary.txt


## Install Required Libraries

In [2]:
!pip -q install -U openai gradio python-dotenv pypdf


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 999.8/999.8 kB 24.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.5/63.5 MB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.5/323.5 kB 13.5 MB/s eta 0:00:00


## Define Tool Functions (students & feedback logs)

In [3]:
from pathlib import Path
import csv, datetime

root = Path("/content")
(logs := root / "logs").mkdir(exist_ok=True)
students_csv = logs / "students.csv"
feedback_csv = logs / "feedback.csv"

if not students_csv.exists():
    with open(students_csv, "w", newline="", encoding="utf-8") as f:
        csv.writer(f).writerow(["ts_iso", "email", "name", "message"])
if not feedback_csv.exists():
    with open(feedback_csv, "w", newline="", encoding="utf-8") as f:
        csv.writer(f).writerow(["ts_iso", "question", "notes"])

def record_student_interest(email: str, name: str, message: str):
    ts = datetime.datetime.utcnow().isoformat()
    with open(students_csv, "a", newline="", encoding="utf-8") as f:
        csv.writer(f).writerow([ts, email, name, message])
    print(f"[student] {ts} | {name} <{email}> - {message}")
    return {"ok": True, "ts": ts}

def record_feedback(question: str):
    ts = datetime.datetime.utcnow().isoformat()
    with open(feedback_csv, "a", newline="", encoding="utf-8") as f:
        csv.writer(f).writerow([ts, question, "model didn't know"])
    print(f"[feedback] {ts} | {question}")
    return {"ok": True, "ts": ts}


## Sample AUB Course Catalog (for realistic answers)

In [4]:
# Small fictional catalog to ground answers
courses = {
    "CMPS 270": "Machine Learning - Introduction to algorithms and model training. Prereq: CMPS 200.",
    "DATA 200": "Introduction to Data Science - Fundamentals of data visualization and analysis.",
    "STAT 233": "Probability and Statistics - Covers probability theory, random variables, and data analysis.",
    "ECON 222": "Econometrics I - Quantitative methods for economics and social sciences.",
    "ENGL 203": "Academic Writing - Advanced academic writing and research composition.",
    "BIOL 201": "Cell and Molecular Biology - Core molecular and cellular biology principles."
}

def list_courses():
    return "\\n".join([f"{k}: {v}" for k,v in courses.items()])


## Load Context and Define System Prompt

In [5]:
import os
from dotenv import load_dotenv
from pypdf import PdfReader

load_dotenv()

SUMMARY_PATH = "/content/me/service_summary.txt"
PDF_PATH = "/content/me/about_service.pdf"

def load_service_context():
    summary, pdf_text = "", ""
    if os.path.exists(SUMMARY_PATH):
        with open(SUMMARY_PATH, "r", encoding="utf-8") as f:
            summary = f.read()
    if os.path.exists(PDF_PATH):
        try:
            reader = PdfReader(PDF_PATH)
            for page in reader.pages:
                pdf_text += page.extract_text() + "\\n"
        except Exception as e:
            pdf_text = "(PDF text unavailable: " + str(e) + ")"
    return summary, pdf_text

SYSTEM_PROMPT = (
    "You are AUB Compass, the official tutoring and course-advising assistant for the American University of Beirut (AUB). "
    "You help students explore courses, understand prerequisites, plan study paths, and find tutoring help. "
    "Keep your tone professional, encouraging, and factual. "
    "Base answers on the AUB course catalog provided. "
    "If you cannot answer, call the `record_feedback` tool with the exact question. "
    "If a student wants tutoring or course planning help, collect their name and email and call `record_student_interest`. "
    "Always encourage students to confirm plans with official AUB advisors before registration."
)


## Define the Agent Logic and Tool Integration

In [ ]:
from openai import OpenAI
import json
# insert your OpenAI API key here
api_key = ""
client = OpenAI(api_key=api_key)
tools = [
    {
        "type": "function",
        "function": {
            "name": "record_student_interest",
            "description": "Store a student's info and request so the tutoring team can follow up.",
            "parameters": {
                "type": "object",
                "properties": {
                    "email": {"type": "string", "description": "Student email"},
                    "name": {"type": "string", "description": "Student name"},
                    "message": {"type": "string", "description": "Short summary of their tutoring/course question"}
                },
                "required": ["email", "name", "message"]
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "record_feedback",
            "description": "Log an unanswered or out-of-scope question for improvement.",
            "parameters": {
                "type": "object",
                "properties": {
                    "question": {"type": "string", "description": "The question that couldn't be answered"}
                },
                "required": ["question"]
            },
        },
    },
]

def run_agent(user_text, history):
    summary, pdf_text = load_service_context()
    course_block = "AUB Courses (sample)\\n" + "\\n".join([f"- {k}: {v}" for k,v in courses.items()])
    messages = [
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "system", "content": f"SUMMARY:\\n{summary}"},
        {"role": "system", "content": f"PDF:\\n{pdf_text}"},
        {"role": "system", "content": course_block},
    ]
    for role, content in history:
        messages.append({"role": role, "content": content})
    messages.append({"role": "user", "content": user_text})

    resp = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages,
        tools=tools,
        tool_choice="auto",
        temperature=0.2,
    )

    msg = resp.choices[0].message
    if getattr(msg, 'tool_calls', None):
        tool_msgs = []
        for call in msg.tool_calls:
            name = call.function.name
            args = json.loads(call.function.arguments or "{}")
            if name == "record_student_interest":
                result = record_student_interest(**args)
            elif name == "record_feedback":
                result = record_feedback(**args)
            else:
                result = {"ok": False, "error": f"Unknown tool {name}"}
            tool_msgs.append({"role": "tool", "tool_call_id": call.id, "name": name, "content": json.dumps(result)})
        follow = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=messages + [{"role": "assistant", "tool_calls": msg.tool_calls}] + tool_msgs
        )
        return follow.choices[0].message.content
    return msg.content


## Launch the Gradio Chat Interface

In [8]:
import gradio as gr

with gr.Blocks(title="AUB Compass – AUB Tutoring & Course Advisor") as demo:
    gr.Markdown("## AUB Compass – Intelligent Tutoring & Course Advisor")
    gr.Markdown("Ask about AUB classes, tutoring, or course recommendations. (Sample catalog included.)")
    chat = gr.Chatbot()
    state = gr.State([])

    def respond(user, history, st):
        linear = []
        for u,a in history:
            if u: linear.append(("user", u))
            if a: linear.append(("assistant", a))
        reply = run_agent(user, linear)
        history.append([user, reply])
        st.append(("user", user)); st.append(("assistant", reply))
        return history, st

    inp = gr.Textbox(placeholder="Ask about AUB classes, tutoring, or course recommendations...", label="Message")
    gr.ClearButton([inp, chat])
    inp.submit(respond, [inp, chat, state], [chat, state])

# Uncomment to launch in Colab
demo.launch()
print("Ready. To launch the UI in Colab, call demo.launch().")


/tmp/ipython-input-3133211268.py:6: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chat = gr.Chatbot()


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://4e6b10b1e723f4bfed.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Ready. To launch the UI in Colab, call demo.launch().


## Realistic Example: Course Recommendation

**User:** I'm a sophomore in Computer Science at AUB. Which electives help with data skills?

**AUB Compass:** You could consider **DATA 200 – Introduction to Data Science** and **STAT 233 – Probability and Statistics**. If you've completed **CMPS 200**, you may also take **CMPS 270 – Machine Learning** next semester.